## Advanced Box Score Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a game's play by play data and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [71]:
import pandas as pd

pbp_data = pd.read_json('data/pbp/pbp-data-pitt-duke.json')
# pbp_data.info()

drives = pd.read_json('data/drives/drive-data-pitt-duke.json')
drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','game_id'], axis = 1, inplace=True) 
drives = drives[
    ~drives.drive_result.isin(['END OF HALF','END OF GAME'])
]

away_team = 'Pittsburgh'
home_team = 'Duke'

In [72]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives.loc[
    drives.offense == away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))

# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]
print("Total clean plays:", len(pbp_data))

Total Drives: 37
Total Plays: 223
Garbage Time Plays: 3
Non-Garbage Time Plays: 220
Total clean plays: 172


In [73]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    elif ((down == 4) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)

In [74]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

print("Success Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

Success Rates
Pittsburgh
Overall: 0.3373493975903614
Passing: 0.3404255319148936
Rushing: 0.3333333333333333
Duke
Overall: 0.3146067415730337
Passing: 0.2
Rushing: 0.4318181818181818


In [75]:
# Explosiveness rates
# Defined as rate of 15+ yards gains

print("Explosiveness Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

Explosiveness Rates
Pittsburgh
Overall: 0.10843373493975904
Passing: 0.14893617021276595
Rushing: 0.05555555555555555
Duke
Overall: 0.07865168539325842
Passing: 0.13333333333333333
Rushing: 0.022727272727272728


In [76]:
# Havoc plays
havoc_plays = pbp_data[
    (((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (pbp_data.play_type == 'Sack')
    | (pbp_data.play_type.str.contains('Interception', regex=False))
    | (pbp_data.yards_gained < 0))
    & (pbp_data.play_type != 'Penalty')
]

print(away_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == away_team
]), "/", len(pbp_data[
    pbp_data.defense == away_team
]), "(",len(havoc_plays[
    havoc_plays.defense == away_team
]) / len(pbp_data[
    pbp_data.defense == away_team
]),")")

print(home_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == home_team
]), "/", len(pbp_data[
    pbp_data.defense == home_team
]), "(",len(havoc_plays[
    havoc_plays.defense == home_team
]) / len(pbp_data[
    pbp_data.defense == home_team
]),")")

Pittsburgh Havoc Rate:  17 / 89 ( 0.19101123595505617 )
Duke Havoc Rate:  18 / 83 ( 0.21686746987951808 )


In [77]:
# Calculating converted 3rd and 8+s
ignore_turnovers = ['Fumble Recovery (Opponent)','Fumble Return Touchdown',"Interception","Pass Interception Return","Interception Return Touchdown"]

third_longs = pbp_data[
    (pbp_data.down == 3)
    & (pbp_data.distance >= 8)
    & (~pbp_data.play_type.isin(ignore_turnovers))
]
# converted_3rd_longs

away_team_thirds = third_longs[
    (third_longs.offense == away_team)
]

away_team_thirds_converted = away_team_thirds[
    (away_team_thirds.yards_gained >= 8)
]
print("Converted 3rd and 8+ for", away_team,":",len(away_team_thirds_converted),"of",len(away_team_thirds))

home_team_thirds = third_longs[
    (third_longs.offense == home_team)
]

home_team_thirds_converted = home_team_thirds[
    (home_team_thirds.yards_gained >= 8)
]

print("Converted 3rd and 8+ for", home_team,":",len(home_team_thirds_converted),"of",len(home_team_thirds))

Converted 3rd and 8+ for Pittsburgh : 2 of 9
Converted 3rd and 8+ for Duke : 3 of 7


In [99]:
away_team_drives = drives[
    drives.offense == away_team
]

print(away_team, "Drives:", len(away_team_drives))
print(away_team,"Yards:",sum(away_team_drives.yards))
print(away_team,"Plays:",sum(away_team_drives.plays))
print(away_team, "Avg Starting Field Position:", sum(away_team_drives.start_yardline) / len(away_team_drives))
print(away_team, "Yards per Play:", sum(away_team_drives.yards) / sum(away_team_drives.plays))
print(away_team, "Plays per Drive:", sum(away_team_drives.plays) / len(away_team_drives))
print(away_team, "Yards per Drive:", sum(away_team_drives.yards) / len(away_team_drives))
away_team_drives

Pittsburgh Drives: 18
Pittsburgh Yards: 347
Pittsburgh Plays: 85
Pittsburgh Avg Starting Field Position: 40.55555555555556
Pittsburgh Yards per Play: 4.08235294117647
Pittsburgh Plays per Drive: 4.722222222222222
Pittsburgh Yards per Drive: 19.27777777777778


,defense,drive_result,end_period,end_yardline,id,offense,plays,scoring,start_period,start_yardline,yards
0,Duke,PUNT,1,46,4011124741,Pittsburgh,6,False,1,25,21
2,Duke,PUNT,1,23,4011124743,Pittsburgh,3,False,1,17,6
4,Duke,PUNT,1,54,4011124745,Pittsburgh,3,False,1,51,3
7,Duke,FG,1,75,4011124749,Pittsburgh,8,True,1,66,9
9,Duke,INT,2,91,40111247411,Pittsburgh,16,False,2,23,68
11,Duke,FG,2,80,40111247413,Pittsburgh,8,True,2,55,25
13,Duke,TD,2,100,40111247415,Pittsburgh,1,True,2,81,19
15,Duke,DOWNS,2,53,40111247417,Pittsburgh,9,False,2,11,42
17,Duke,PUNT,3,53,40111247420,Pittsburgh,4,False,3,28,25
19,Duke,TD,3,100,40111247422,Pittsburgh,2,True,3,93,7


In [100]:
home_team_drives = drives[
    drives.offense == home_team
]

print(home_team, "Drives:", len(home_team_drives))
print(home_team,"Yards:",sum(home_team_drives.yards))
print(home_team,"Plays:",sum(home_team_drives.plays))
print(home_team, "Avg Starting Field Position:", sum(home_team_drives.start_yardline) / len(home_team_drives))
print(home_team, "Yards per Play:", sum(home_team_drives.yards) / sum(home_team_drives.plays))
print(home_team, "Plays per Drive:", sum(home_team_drives.plays) / len(home_team_drives))
print(home_team, "Yards per Drive:", sum(home_team_drives.yards) / len(home_team_drives))
home_team_drives

Duke Drives: 19
Duke Yards: 331
Duke Plays: 90
Duke Avg Starting Field Position: 31.473684210526315
Duke Yards per Play: 3.6777777777777776
Duke Plays per Drive: 4.7368421052631575
Duke Yards per Drive: 17.42105263157895


,defense,drive_result,end_period,end_yardline,id,offense,plays,scoring,start_period,start_yardline,yards
1,Pittsburgh,FG,1,85,4011124742,Duke,14,True,1,20,65
3,Pittsburgh,FUMBLE,1,49,4011124744,Duke,3,False,1,38,11
5,Pittsburgh,INT TD,1,18,4011124746,Duke,1,True,1,18,0
6,Pittsburgh,INT,1,25,4011124748,Duke,2,False,1,23,2
8,Pittsburgh,PUNT,2,38,40111247410,Duke,6,False,1,25,13
10,Pittsburgh,PUNT,2,13,40111247412,Duke,3,False,2,9,4
12,Pittsburgh,PUNT,2,26,40111247414,Duke,3,False,2,22,4
14,Pittsburgh,PUNT,2,31,40111247416,Duke,3,False,2,25,6
16,Pittsburgh,PUNT,3,30,40111247419,Duke,3,False,3,23,7
18,Pittsburgh,FUMBLE,3,7,40111247421,Duke,2,False,3,15,-8


In [80]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [81]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = pbp_data[
    pbp_data.play_type.isin(pass_types)
]
rush_plays = pbp_data[
    pbp_data.play_type.isin(rush_types)
]

In [82]:
print(away_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == away_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == away_team)
]))

Pittsburgh Success Rate on Std Downs: 0.4


In [83]:
print(away_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == away_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == away_team)
]))

Pittsburgh Success Rate on Passing Downs: 0.21428571428571427


In [84]:
print(home_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == home_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == home_team)
]))

Duke Success Rate on Std Downs: 0.40350877192982454


In [85]:
print(home_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == home_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == home_team)
]))

Duke Success Rate on Passing Downs: 0.15625


In [86]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

In [87]:
away_team_stuffs = stuffed_plays[
    stuffed_plays.defense == away_team
]
print(away_team,"Defensive Stuff Rate: ",len(away_team_stuffs),"/",len(rush_plays[rush_plays.defense == away_team]),"(",len(away_team_stuffs)/len(rush_plays[rush_plays.defense == away_team]),")")

Pittsburgh Defensive Stuff Rate:  4 / 44 ( 0.09090909090909091 )


In [88]:
home_team_stuffs = stuffed_plays[
    stuffed_plays.defense == home_team
]
print(home_team,"Defensive Stuff Rate: ",len(home_team_stuffs),"/",len(rush_plays[rush_plays.defense == home_team]),"(",len(home_team_stuffs)/len(rush_plays[rush_plays.defense == home_team]),")")

Duke Defensive Stuff Rate:  7 / 36 ( 0.19444444444444445 )


In [89]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 5)
]
print(away_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == away_team]),"/",len(rush_plays[rush_plays.offense == away_team]),"(",len(rush_opps[rush_opps.offense == away_team])/len(rush_plays[rush_plays.offense == away_team]),")")

Pittsburgh Rush Opp Rate:  11 / 36 ( 0.3055555555555556 )


In [90]:
print(home_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == home_team]),"/",len(rush_plays[rush_plays.offense == home_team]),"(",len(rush_opps[rush_opps.offense == home_team])/len(rush_plays[rush_plays.offense == home_team]),")")

Duke Rush Opp Rate:  11 / 44 ( 0.25 )


In [91]:
# Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif (yards_gained >= 0 & yards_gained <= 3):
        return yards_gained * 1.0
    elif (yards_gained >= 4 & yards_gained <= 6):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)
print(away_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].line_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].highlight_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].yards_gained) / len(rush_plays[rush_plays.offense == away_team]))
print(home_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].line_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].highlight_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].yards_gained) / len(rush_plays[rush_plays.offense == home_team]))

Pittsburgh Line Yards per Carry: 2.3333333333333335
Pittsburgh Highlight Yards per Carry: 0.3611111111111111
Pittsburgh Yards per Carry: 2.6944444444444446
Duke Line Yards per Carry: 2.5511363636363638
Duke Highlight Yards per Carry: 0.1534090909090909
Duke Yards per Carry: 2.7045454545454546


/Users/aeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/aeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [228]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = pbp_data[
    (pbp_data.play_type.str.contains('Interception', regex=False))
    | ((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.5 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# away_team Adj Turnovers
away_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == away_team)
    | (adj_turnover_plays.defense == away_team)
]

away_team_ints_def = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_ints_off = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_pds = len(away_team_tos[
   (away_team_tos.play_type == 'Pass Incompletion')
    & (away_team_tos.play_text.str.contains('broken up', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_fum_rec = away_team_tos[((away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.defense == away_team))]
away_team_fum_lost = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.offense == away_team)]

In [241]:
# home_team Adj Turnovers (broken)
home_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == home_team)
    | (adj_turnover_plays.defense == home_team)
]

home_team_ints_def = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_ints_off = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_pds = len(home_team_tos[
   (home_team_tos.play_type == 'Pass Incompletion')
    & (home_team_tos.play_text.str.contains('broken up', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_fum_rec = home_team_tos[((home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.defense == home_team))]
home_team_fum_lost = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.offense == home_team)]

print(away_team)
print("Def INTs:",away_team_ints_def)
print("Off INTs:",away_team_ints_off)
print("Def PDs:",(away_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (away_team_pds + away_team_ints_def))
print("Fum Recovered:",len(away_team_fum_rec))
print("Fum Lost:",len(away_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Actual TO:",away_team_ints_off + len(away_team_fum_lost))
print("Actual TO Margin:",-1 * away_team_ints_off - len(away_team_fum_lost) + away_team_ints_def + len(away_team_fum_rec))
away_team_exp_to = 0.22 * (away_team_pds + away_team_ints_def) + adj_fum
print("Exp TO:",away_team_exp_to)

print("")

print(home_team)
print("Def INTs:",home_team_ints_def)
print("Off INTs:",home_team_ints_off)
print("Def PDs:",(home_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (home_team_pds + home_team_ints_def))
print("Fum Recovered:",len(home_team_fum_rec))
print("Fum Lost:",len(home_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Actual TO:",home_team_ints_off + len(home_team_fum_lost))
print("Actual TO Margin:",-1 * home_team_ints_off - len(home_team_fum_lost) + home_team_ints_def + len(home_team_fum_rec))
home_team_exp_to = 0.22 * (home_team_pds + home_team_ints_def) + adj_fum
print("Exp TO:",home_team_exp_to)

Pittsburgh
Def INTs: 3
Off INTs: 2
Def PDs: 11
Exp INTs: 2.42
Fum Recovered: 1
Fum Lost: 1
Exp Fum: 1.0
Actual TO: 3
Actual TO Margin: 1
Exp TO: 3.42

Duke
Def INTs: 2
Off INTs: 3
Def PDs: 6
Exp INTs: 1.1
Fum Recovered: 1
Fum Lost: 1
Exp Fum: 1.0
Actual TO: 4
Actual TO Margin: -1
Exp TO: 2.1


In [242]:
# home_team_tos

In [249]:
away_team_exp_to_margin = (away_team_exp_to - home_team_exp_to)
print("Exp TO Margin for",away_team,":", away_team_exp_to_margin)
home_team_exp_to_margin = (home_team_exp_to - away_team_exp_to)
print("Exp TO Margin for",home_team,":", home_team_exp_to_margin)

exp_to_margin = (0.22 * ((away_team_pds + away_team_ints_def) + (home_team_pds + home_team_ints_def))) + adj_fum
exp_to_margin

Exp TO Margin for Pittsburgh : 1.3199999999999998
Exp TO Margin for Duke : -1.3199999999999998


4.52

In [250]:
print("TO Luck (pts) for",away_team,":", (away_team_ints_def + len(away_team_fum_rec) - away_team_ints_off - len(away_team_fum_lost) - exp_to_margin) * 5.0)
print("TO Luck (pts) for",home_team,":", (home_team_ints_def + len(home_team_fum_rec) - home_team_ints_off - len(home_team_fum_lost) - exp_to_margin) * 5.0)


TO Luck (pts) for Pittsburgh : -17.599999999999998
TO Luck (pts) for Duke : -27.599999999999998


In [96]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 35, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 65)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 35)")
print(away_team)
away_team_scoring_opps = scoring_opps[
    scoring_opps.offense == away_team
]
print("Total:",len(away_team_scoring_opps))
print("Scored:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]) / len(away_team_scoring_opps))
print("Opps/Drive:",len(away_team_scoring_opps) / len(drives[drives.offense == away_team]))
print("Points/Opp:",(len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'TD']) * 7 + len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'FG'] * 3)) / len(away_team_scoring_opps))

print("")
# home team's scoring opps
print(home_team)
home_team_scoring_opps = scoring_opps[
    scoring_opps.offense == home_team
]
print("Total:",len(home_team_scoring_opps))
print("Scored:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]) / len(home_team_scoring_opps))
print("Opps/Drive:",len(home_team_scoring_opps) / len(drives[drives.offense == home_team]))
print("Points/Opp:",(len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'TD']) * 7 + len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'FG'] * 3)) / len(home_team_scoring_opps))

Scoring Opportunities (IE: Drives inside Opponent's 35)
Pittsburgh
Total: 7
Scored: 5
Opp Efficiency: 0.7142857142857143
Opps/Drive: 0.3888888888888889
Points/Opp: 3.2857142857142856

Duke
Total: 5
Scored: 5
Opp Efficiency: 1.0
Opps/Drive: 0.2631578947368421
Points/Opp: 5.8


In [97]:
# Rushing Stats
rush_carries = rush_plays[
    (~rush_plays.play_type.isin(['Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']))
]

# look these up manually
away_team_rushers = ["Kenny Pickett","Todd Sibley Jr.","V'Lique Carter","Shocky Jacques-Louis","Maurice Ffrench"]
home_team_rushers = ["Quentin Harris","Deon Jackson","Mataeo Durant","Jordan Waters"]

def verify_division(num1, num2):
    if num2 == 0:
        return 0
    else:
        return num1 / num2

print(away_team,"Rushing Stats")
for qb in away_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True)]), len(pbp_data[(pbp_data.play_text.str.contains(qb))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")

print("")

print(home_team,"Rushing Stats")
for qb in home_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True)]), len(pbp_data[(pbp_data.play_text.str.contains(qb))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")


Pittsburgh Rushing Stats
Kenny Pickett: 3 car 11 yards 0 TDs 1 Fum ( 3.6666666666666665 YPC , 3.6666666666666665 line YPC , SR%: 0.3333333333333333 , Opp Rate: 0.6666666666666666 , Hlt/Opp: 5.5 )
Todd Sibley Jr.: 12 car 75 yards 0 TDs 0 Fum ( 6.25 YPC , 6.25 line YPC , SR%: 0.23076923076923078 , Opp Rate: 0.3333333333333333 , Hlt/Opp: 18.75 )
V'Lique Carter: 13 car 18 yards 0 TDs 0 Fum ( 1.3846153846153846 YPC , 1.0192307692307692 line YPC , SR%: 0.375 , Opp Rate: 0.3076923076923077 , Hlt/Opp: 4.5 )
Shocky Jacques-Louis: 1 car 10 yards 0 TDs 0 Fum ( 10.0 YPC , 10.0 line YPC , SR%: 0.6666666666666666 , Opp Rate: 1.0 , Hlt/Opp: 10.0 )
Maurice Ffrench: 2 car 0 yards 0 TDs 0 Fum ( 0.0 YPC , -0.375 line YPC , SR%: 0.35714285714285715 , Opp Rate: 0.0 , Hlt/Opp: 0 )

Duke Rushing Stats
Quentin Harris: 14 car 52 yards 2 TDs 1 Fum ( 3.7142857142857144 YPC , 3.7142857142857144 line YPC , SR%: 0.26229508196721313 , Opp Rate: 0.2857142857142857 , Hlt/Opp: 13.0 )
Deon Jackson: 19 car 60 yards 1 TDs

In [98]:
# Passing Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
]

# look these up manually
away_team_qbs = ["Kenny Pickett"]
home_team_qbs = ["Quentin Harris"]

print(away_team,"Passing Stats")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")
print("")
print(home_team,"Passing Stats")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")


Pittsburgh Passing Stats
Kenny Pickett: 28 / 47 225 yards 3 TDs 2 INTs

Duke Passing Stats
Quentin Harris: 18 / 43 180 yards 1 TDs 2 INTs
